# CORD-19-collect-scopus-data

In general, this jupyter notebook is designated to collect additional data via scopus to enbroaden the CORD19 dataset: 
https://datadryad.org/stash/dataset/doi:10.5061/dryad.vmcvdncs0

First, relevant packages must be imported to the Notebook.

In [1]:
import numpy as np
import pandas as pd
import csv
import ast
import collections
import matplotlib.pyplot as plt
import Levenshtein as lev
from fuzzywuzzy import fuzz 
import datetime
import matplotlib.pyplot as plt
import re
from urllib.parse import urlparse
from collections import Counter

from elsapy.elsclient import ElsClient
from elsapy.elsdoc import FullDoc, AbsDoc
from elsapy.elssearch import ElsSearch

import time # for sleep
from pybtex.database import parse_file, BibliographyData, Entry
import json
from elsapy.elsclient import ElsClient
from elsapy.elsdoc import AbsDoc
from elsapy.elssearch import ElsSearch

Get the data and save it to a variable.

In [2]:
CORD19_CSV = pd.read_csv('../data/cord-19/CORD19_software_mentions.csv')

Check the length of the column containing doi's.

In [3]:
len(CORD19_CSV['doi'])

77448

Display the column doi to see if there are inconsistencies such as NaN's

In [4]:
doi = CORD19_CSV['doi']
doi

0                                 NaN
1          10.1016/j.regg.2021.01.002
2           10.1016/j.rec.2020.08.002
3        10.1016/j.vetmic.2006.11.026
4                   10.3390/v12080849
                     ...             
77443      10.1007/s11229-020-02869-9
77444                             NaN
77445     10.1101/2020.05.13.20100206
77446      10.1007/s42991-020-00052-8
77447     10.1101/2020.09.14.20194670
Name: doi, Length: 77448, dtype: object

Create a series with solely unique values and neglect NaN's. It is important to sort the unique values. Otherwise, the method is creating different results after each restart of the notebook. 

In [5]:
doi_counted = doi.value_counts().sort_index(ascending=True)
doi_counted

10.1001/jamainternmed.2020.1369       1
10.1001/jamanetworkopen.2020.16382    1
10.1001/jamanetworkopen.2020.17521    1
10.1001/jamanetworkopen.2020.20485    1
10.1001/jamanetworkopen.2020.24984    1
                                     ..
10.9745/ghsp-d-20-00115               1
10.9745/ghsp-d-20-00171               1
10.9745/ghsp-d-20-00218               1
10.9758/cpn.2020.18.4.607             1
10.9781/ijimai.2020.02.002            1
Name: doi, Length: 74302, dtype: int64

The following function determines the requested information from the Scopus API. (https://api.elsevier.com/content/search/scopus?query=DOI(10.1109/MCOM.2016.7509373)&apiKey=6d485ef1fe1408712f37e8a783a285a4)

In [6]:
#Adapted from https://github.com/ElsevierDev/elsapy/blob/master/exampleProg.py
def fetch_scopus_api(client, doi):
    """obtain additional paper information from scopus by doi
    """
    doc_srch = ElsSearch("DOI("+doi+")",'scopus')
    doc_srch.execute(client, get_all = True)
    #print ("doc_srch has", len(doc_srch.results), "results.")
    #print(doc_srch.results)
    try:
        scopus_id=doc_srch.results[0]["dc:identifier"].split(":")[1]
        scp_doc = AbsDoc(scp_id = scopus_id)
        if scp_doc.read(client):
            # print ("scp_doc.title: ", scp_doc.title)
            scp_doc.write()   
        else:
            print ("Read document failed.")
        # print(scp_doc.data["affiliation"])
        return scp_doc.data
    except:
        return None

Thusly, the configuration file is set up and contains an APIkey. Further information: https://github.com/ElsevierDev/elsapy/blob/master/CONFIG.md

In [7]:
con_file = open("config.json")
config = json.load(con_file)
con_file.close()

Moreover, the client is initialized with the API-Key.

In [8]:
client = ElsClient(config['apikey'])

For demonstation purposes, the following cells shows which data is returned by the Scopus API. 

In [9]:
return_example = fetch_scopus_api(client, '10.1016/j.dsx.2020.04.012')
print(json.dumps(return_example, indent=2))

{
  "affiliation": [
    {
      "affiliation-city": "New Delhi",
      "affilname": "Jamia Hamdard",
      "affiliation-country": "India"
    },
    {
      "affiliation-city": "New Delhi",
      "affilname": "Jamia Millia Islamia",
      "affiliation-country": "India"
    },
    {
      "affiliation-city": "New Delhi",
      "affilname": "Indraprastha Apollo Hospitals",
      "affiliation-country": "India"
    }
  ],
  "coredata": {
    "srctype": "j",
    "eid": "2-s2.0-85083171050",
    "pubmed-id": "32305024",
    "prism:coverDate": "2020-07-01",
    "prism:aggregationType": "Journal",
    "prism:url": "https://api.elsevier.com/content/abstract/scopus_id/85083171050",
    "dc:creator": {
      "author": [
        {
          "ce:given-name": "Raju",
          "preferred-name": {
            "ce:given-name": "Raju",
            "ce:initials": "R.",
            "ce:surname": "Vaishya",
            "ce:indexed-name": "Vaishya R."
          },
          "@seq": "1",
          "ce:init

Based on the returned data, further analysis is conductable. Therefore, two notebooks are created to analyse data linked to: 
<ul>
  <li>affiliation</li>
  <li>coredata</li>
</ul>    

Thusly, the already fetched coredata and affiliation are read and combined to a DataFrame for further processing.

In [10]:
df_current_extra_info = pd.DataFrame()
try:
    read_affiliation = pd.read_pickle('extra_info_affiliation_CS.pkl')
    read_coredata = pd.read_pickle('extra_info_coredata_CS.pkl')
    df_current_extra_info['affiliation'] = read_affiliation
    df_current_extra_info['coredata'] = read_coredata
    df_current_extra_info
except:
    print("The DataFrame is empty")
    #if the dataframe is not empty set the variable to show the dataframe

The length of the DataFrame containing the current information is assigned to a variable to be used for further processing. 
Therefore, the length will be used within a while loop as a starting index. 

In [11]:
len_df_current_extra_info = len(df_current_extra_info)
len_df_current_extra_info

54529

In [12]:
df_current_extra_info

,affiliation,coredata
0,"[{'affiliation-city': 'Palo Alto', 'affilname'...","{'srctype': 'j', 'eid': '2-s2.0-85083266658', ..."
1,"[{'affiliation-city': 'Seattle', 'affilname': ...","{'srctype': 'j', 'prism:issueIdentifier': '7',..."
2,"[{'affiliation-city': 'Cambridge', 'affilname'...","{'srctype': 'j', 'prism:issueIdentifier': '8',..."
3,"[{'affiliation-city': 'Madison', 'affilname': ...","{'srctype': 'j', 'prism:issueIdentifier': '9',..."
4,"[{'affiliation-city': 'Los Angeles', 'affilnam...","{'srctype': 'j', 'prism:issueIdentifier': '11'..."
...,...,...
54524,"[{'affiliation-city': 'Oslo', 'affilname': 'Ul...","{'srctype': 'j', 'prism:issueIdentifier': '4',..."
54525,"[{'affiliation-city': 'Innsbruck', 'affilname'...","{'srctype': 'j', 'prism:issueIdentifier': '4',..."
54526,"[{'affiliation-city': 'London', 'affilname': '...","{'srctype': 'j', 'prism:issueIdentifier': '6',..."
54527,"[{'affiliation-city': 'Cambridge', 'affilname'...","{'srctype': 'j', 'prism:issueIdentifier': '1',..."


In [13]:
def contains_only_None(dic):
    """
    This functions inspects an dictionary and returns True if it solely contains None values
    """
    return len(dic) == sum(value == None for value in dict_new_extra_info.values())

In [14]:
def append_fetched_data_to_df(df_current_extra_info, dic):
    """
    This function appends or inserts newly fetched data to the DataFrame containing scopus data.
    """
    #df_current_extra_info -> holding the latest data, new data needs to be appended to it, 
    #df_newly_fetched_transposed -> holdy newly fetched data, needs to be inserted or fetched
    
    if contains_only_None(dic):
        placeholder_entries = pd.DataFrame(np.empty((len(dict_new_extra_info),2),dtype=object),columns=['affiliation','coredata'], index=dict_new_extra_info.keys())
        df_newly_fetched_transposed = placeholder_entries
        print(placeholder_entries)
    else:
        #Prior appending, the dictionary is transformed to a DataFrame
        df_newly_fetched = pd.DataFrame(dic)
        #For readability, the DataFrame is transposed
        df_newly_fetched_transposed = df_newly_fetched.T
        print(df_newly_fetched_transposed)
    
    #Insert newly fetched rows which were previously not successful appended
    for index, row in df_newly_fetched_transposed.iterrows():
        #insert to current extra info DataFrame because the row is existent
        if index in df_current_extra_info.index and row.affiliation is not None:
            df_current_extra_info.loc[index] = row
        #append to current extra info DataFrame because the row is new     
        if index not in df_current_extra_info.index:
            df_current_extra_info = df_current_extra_info.append(row, ignore_index=True)
            
    #returning DataFrame with inserted and replaced rows. 
    return df_current_extra_info

Both Dataframes columns are stored each to an object. The series objects are stored to each to a pkl-file which is not exceeding the size of 100MB allowing GitHub uploads.

In [15]:
def store_df_columns(df):
    ser_affiliation = df['affiliation']
    ser_coredata = df['coredata']
    ser_affiliation.to_pickle('extra_info_affiliation_CS.pkl')
    ser_coredata.to_pickle('extra_info_coredata_CS.pkl')
    return ser_affiliation, ser_coredata

In [16]:
# placeholder_entries = pd.DataFrame(np.empty((4,2),dtype=object),columns=['affiliation','coredata'])

In [17]:
# placeholder_entries

Subsequently, the fetched scopus data is stored within a dictionary. Besides, the print function is used to show the state of the process by displaying the latest fetched information. 

In [ ]:
%%time
dict_new_extra_info = dict()
len_dois = len(doi_counted)
def trigger_fetching():
    threshold = 0 
    i = len_df_current_extra_info
    while i < len_dois: #-> upto modified, normally len_dois
        dict_new_extra_info[i] = fetch_scopus_api(client, doi_counted.index[i])
        print("Position fetched: " + str(i) + " -> " +  doi_counted.index[i])
        i = i + 1 
        threshold = threshold + 1
        if threshold > 99:
            df_combined_extra_info = append_fetched_data_to_df(df_current_extra_info, dict_new_extra_info)
            stored_series = store_df_columns(df_combined_extra_info)
            threshold = 0
            print("batch saved")
trigger_fetching()

Position fetched: 54529 -> 10.1183/13993003.03730-2020
Position fetched: 54530 -> 10.1183/13993003.03961-2020
Position fetched: 54531 -> 10.1183/13993003.04116-2020
Position fetched: 54532 -> 10.1183/16000617.0060-2020
Position fetched: 54533 -> 10.1183/16000617.0181-2020
Position fetched: 54534 -> 10.1183/16000617.0240-2020
Position fetched: 54535 -> 10.1183/16000617.0287-2020
Position fetched: 54536 -> 10.1183/16000617.0310-2020
Position fetched: 54537 -> 10.1183/20734735.019317
Position fetched: 54538 -> 10.1183/23120541.00015-2015
Position fetched: 54539 -> 10.1183/23120541.00066-2018
Position fetched: 54540 -> 10.1183/23120541.00096-2018
Position fetched: 54541 -> 10.1183/23120541.00142-2020
Position fetched: 54542 -> 10.1183/23120541.00156-2017
Position fetched: 54543 -> 10.1183/23120541.00174-2020
Position fetched: 54544 -> 10.1183/23120541.00238-2020
Position fetched: 54545 -> 10.1183/23120541.00260-2020
Position fetched: 54546 -> 10.1183/23120541.00304-2020
Position fetched: 5

Position fetched: 54658 -> 10.1186/1471-2105-5-65
Position fetched: 54659 -> 10.1186/1471-2105-5-72
Position fetched: 54660 -> 10.1186/1471-2105-5-96
Position fetched: 54661 -> 10.1186/1471-2105-6-113
Position fetched: 54662 -> 10.1186/1471-2105-6-190
Position fetched: 54663 -> 10.1186/1471-2105-6-23
Position fetched: 54664 -> 10.1186/1471-2105-6-90
Position fetched: 54665 -> 10.1186/1471-2105-7-182
Position fetched: 54666 -> 10.1186/1471-2105-7-232
Position fetched: 54667 -> 10.1186/1471-2105-7-451
Position fetched: 54668 -> 10.1186/1471-2105-7-9
Position fetched: 54669 -> 10.1186/1471-2105-7-s5-s12
Position fetched: 54670 -> 10.1186/1471-2105-8-194
Position fetched: 54671 -> 10.1186/1471-2105-8-21
Position fetched: 54672 -> 10.1186/1471-2105-9-155
Position fetched: 54673 -> 10.1186/1471-2105-9-159
Position fetched: 54674 -> 10.1186/1471-2105-9-304
Position fetched: 54675 -> 10.1186/1471-2105-9-368
Position fetched: 54676 -> 10.1186/1471-2105-9-s1-s19
Position fetched: 54677 -> 10.118

Position fetched: 54790 -> 10.1186/1471-2334-1-6
Position fetched: 54791 -> 10.1186/1471-2334-10-106
Position fetched: 54792 -> 10.1186/1471-2334-10-113
Position fetched: 54793 -> 10.1186/1471-2334-10-136
Position fetched: 54794 -> 10.1186/1471-2334-10-139
Position fetched: 54795 -> 10.1186/1471-2334-10-145
Position fetched: 54796 -> 10.1186/1471-2334-10-215
Position fetched: 54797 -> 10.1186/1471-2334-10-222
Position fetched: 54798 -> 10.1186/1471-2334-10-234
Position fetched: 54799 -> 10.1186/1471-2334-10-256
Position fetched: 54800 -> 10.1186/1471-2334-10-286
Position fetched: 54801 -> 10.1186/1471-2334-10-288
Position fetched: 54802 -> 10.1186/1471-2334-10-296
Position fetched: 54803 -> 10.1186/1471-2334-10-322
Position fetched: 54804 -> 10.1186/1471-2334-10-330
Position fetched: 54805 -> 10.1186/1471-2334-10-35
Position fetched: 54806 -> 10.1186/1471-2334-10-42
Position fetched: 54807 -> 10.1186/1471-2334-10-54
Position fetched: 54808 -> 10.1186/1471-2334-10-62
Position fetched: 5

Position fetched: 54922 -> 10.1186/1471-2431-12-32
Position fetched: 54923 -> 10.1186/1471-2431-12-83
Position fetched: 54924 -> 10.1186/1471-2431-14-37
Position fetched: 54925 -> 10.1186/1471-244x-14-81
Position fetched: 54926 -> 10.1186/1471-2458-10-130
Position fetched: 54927 -> 10.1186/1471-2458-10-138
Position fetched: 54928 -> 10.1186/1471-2458-10-191
                                             affiliation  \
54529  [{'affiliation-city': 'Melbourne', 'affilname'...   
54530  [{'affiliation-city': 'Berlin', 'affilname': '...   
54531  {'affiliation-city': 'Dundee', 'affilname': 'N...   
54532  {'affiliation-city': 'Chicago', 'affilname': '...   
54533  [{'affiliation-city': 'Irvine', 'affilname': '...   
...                                                  ...   
54924  [{'affiliation-city': 'Toronto', 'affilname': ...   
54925  [{'affiliation-city': 'Jinan', 'affilname': 'S...   
54926  [{'affiliation-city': 'Sydney', 'affilname': '...   
54927  [{'affiliation-city': 'Townsville

batch saved
Position fetched: 55029 -> 10.1186/1472-6807-10-6
Position fetched: 55030 -> 10.1186/1472-6807-13-s1-s3
Position fetched: 55031 -> 10.1186/1472-6807-3-8
Position fetched: 55032 -> 10.1186/1472-6807-5-5
Position fetched: 55033 -> 10.1186/1472-6882-12-10
Position fetched: 55034 -> 10.1186/1472-6882-13-367
Position fetched: 55035 -> 10.1186/1472-6882-14-171
Position fetched: 55036 -> 10.1186/1472-6882-14-273
Position fetched: 55037 -> 10.1186/1472-6882-14-90
Position fetched: 55038 -> 10.1186/1472-6920-14-158
Position fetched: 55039 -> 10.1186/1472-6939-7-12
Position fetched: 55040 -> 10.1186/1472-6947-12-147
Position fetched: 55041 -> 10.1186/1472-6947-12-37
Position fetched: 55042 -> 10.1186/1472-6947-5-17
Position fetched: 55043 -> 10.1186/1472-6947-7-15
Position fetched: 55044 -> 10.1186/1472-6947-7-28
Position fetched: 55045 -> 10.1186/1472-6947-8-35
Position fetched: 55046 -> 10.1186/1472-6955-10-4
Position fetched: 55047 -> 10.1186/1472-6963-12-21
Position fetched: 5504

...

batch saved
Position fetched: 56529 -> 10.1186/s12889-020-10103-x
Position fetched: 56530 -> 10.1186/s12889-020-10105-9
Position fetched: 56531 -> 10.1186/s12889-020-10109-5
Position fetched: 56532 -> 10.1186/s12889-020-10116-6
Position fetched: 56533 -> 10.1186/s12889-020-10125-5
Position fetched: 56534 -> 10.1186/s12889-020-10126-4
Position fetched: 56535 -> 10.1186/s12889-020-10131-7
Position fetched: 56536 -> 10.1186/s12889-020-10136-2
Position fetched: 56537 -> 10.1186/s12889-020-10145-1
Position fetched: 56538 -> 10.1186/s12889-020-10147-z
Position fetched: 56539 -> 10.1186/s12889-020-10153-1
Position fetched: 56540 -> 10.1186/s12889-020-8239-2
Position fetched: 56541 -> 10.1186/s12889-020-8279-7
Position fetched: 56542 -> 10.1186/s12889-020-8359-8
Position fetched: 56543 -> 10.1186/s12889-020-8388-3
Position fetched: 56544 -> 10.1186/s12889-020-8455-9
Position fetched: 56545 -> 10.1186/s12889-021-10165-5
Position fetched: 56546 -> 10.1186/s12889-021-10166-4
Position fetched: 565

Position fetched: 56655 -> 10.1186/s12904-020-00616-y
Position fetched: 56656 -> 10.1186/s12904-020-00617-x
Position fetched: 56657 -> 10.1186/s12904-020-00636-8
Position fetched: 56658 -> 10.1186/s12904-020-00644-8
Position fetched: 56659 -> 10.1186/s12904-020-00652-8
Position fetched: 56660 -> 10.1186/s12904-020-00689-9
Position fetched: 56661 -> 10.1186/s12904-020-00691-1
Position fetched: 56662 -> 10.1186/s12904-020-00692-0
Position fetched: 56663 -> 10.1186/s12904-020-00695-x
Position fetched: 56664 -> 10.1186/s12904-020-00704-z
Position fetched: 56665 -> 10.1186/s12904-021-00711-8
Position fetched: 56666 -> 10.1186/s12905-019-0875-2
Position fetched: 56667 -> 10.1186/s12905-020-01086-3
Position fetched: 56668 -> 10.1186/s12905-020-01115-1
Position fetched: 56669 -> 10.1186/s12905-020-01151-x
Position fetched: 56670 -> 10.1186/s12905-020-01163-7
Position fetched: 56671 -> 10.1186/s12905-021-01177-9
Position fetched: 56672 -> 10.1186/s12906-015-0792-3
Position fetched: 56673 -> 10.

Position fetched: 56781 -> 10.1186/s12911-020-01266-z
Position fetched: 56782 -> 10.1186/s12911-020-01275-y
Position fetched: 56783 -> 10.1186/s12911-020-01281-0
Position fetched: 56784 -> 10.1186/s12911-020-01316-6
Position fetched: 56785 -> 10.1186/s12911-020-01321-9
Position fetched: 56786 -> 10.1186/s12911-020-01322-8
Position fetched: 56787 -> 10.1186/s12911-020-01336-2
Position fetched: 56788 -> 10.1186/s12911-020-01338-0
Position fetched: 56789 -> 10.1186/s12911-020-01344-2
Position fetched: 56790 -> 10.1186/s12911-020-01353-1
Position fetched: 56791 -> 10.1186/s12911-020-01373-x
Position fetched: 56792 -> 10.1186/s12911-020-01374-w
Position fetched: 56793 -> 10.1186/s12911-020-1108-1
Position fetched: 56794 -> 10.1186/s12911-020-1114-3
Position fetched: 56795 -> 10.1186/s12911-021-01394-0
Position fetched: 56796 -> 10.1186/s12912-015-0065-y
Position fetched: 56797 -> 10.1186/s12912-020-00457-3
Position fetched: 56798 -> 10.1186/s12912-020-00481-3
Position fetched: 56799 -> 10.1

Position fetched: 56907 -> 10.1186/s12916-020-01663-1
Position fetched: 56908 -> 10.1186/s12916-020-01670-2
Position fetched: 56909 -> 10.1186/s12916-020-01673-z
Position fetched: 56910 -> 10.1186/s12916-020-01682-y
Position fetched: 56911 -> 10.1186/s12916-020-01685-9
Position fetched: 56912 -> 10.1186/s12916-020-01687-7
Position fetched: 56913 -> 10.1186/s12916-020-01691-x
Position fetched: 56914 -> 10.1186/s12916-020-01692-w
Position fetched: 56915 -> 10.1186/s12916-020-01696-6
Position fetched: 56916 -> 10.1186/s12916-020-01705-8
Position fetched: 56917 -> 10.1186/s12916-020-01712-9
Position fetched: 56918 -> 10.1186/s12916-020-01719-2
Position fetched: 56919 -> 10.1186/s12916-020-01726-3
Position fetched: 56920 -> 10.1186/s12916-020-01731-6
Position fetched: 56921 -> 10.1186/s12916-020-01732-5
Position fetched: 56922 -> 10.1186/s12916-020-01735-2
Position fetched: 56923 -> 10.1186/s12916-020-01739-y
Position fetched: 56924 -> 10.1186/s12916-020-01747-y
Position fetched: 56925 -> 1

batch saved
Position fetched: 57029 -> 10.1186/s12917-018-1640-8
Position fetched: 57030 -> 10.1186/s12917-018-1720-9
Position fetched: 57031 -> 10.1186/s12917-018-1736-1
Position fetched: 57032 -> 10.1186/s12917-018-1756-x
Position fetched: 57033 -> 10.1186/s12917-019-1773-4
Position fetched: 57034 -> 10.1186/s12917-019-1774-3
Position fetched: 57035 -> 10.1186/s12917-019-1796-x
Position fetched: 57036 -> 10.1186/s12917-019-1802-3
Position fetched: 57037 -> 10.1186/s12917-019-1848-2
Position fetched: 57038 -> 10.1186/s12917-019-1851-7
Position fetched: 57039 -> 10.1186/s12917-019-1862-4
Position fetched: 57040 -> 10.1186/s12917-019-1877-x
Position fetched: 57041 -> 10.1186/s12917-019-1887-8
Position fetched: 57042 -> 10.1186/s12917-019-1898-5
Position fetched: 57043 -> 10.1186/s12917-019-1909-6
Position fetched: 57044 -> 10.1186/s12917-019-1911-z
Position fetched: 57045 -> 10.1186/s12917-019-1925-6
Position fetched: 57046 -> 10.1186/s12917-019-1927-4
Position fetched: 57047 -> 10.1186

The following cell is useful when the process above is interrupted. Therefore, the dictionary containing fetched information can be narrowed down to useful entries. 

In [ ]:
# def save_new_extra_info(len_df_current_extra_info, upto):
#     """
#     This function is used to separate successfull API calls from API calls which were prevented due to an invalid API-Key.
#     As a result, this function returns a range of valid entries up to the given parameter. 
#     """
#     dict_new_extra_info_saver = dict()
#     i = len_df_current_extra_info
#     while i < upto:
#         #print("Position: " + str(i) + " -> " +  doi_counted.index[i])
#         dict_new_extra_info_saver[i] = dict_new_extra_info[i]
#         i = i + 1 
#     return dict_new_extra_info_saver

The existing and newly fetched information are combined into one DataFrame. 

In [ ]:
# df_combined_extra_info = append_fetched_data_to_df(df_current_extra_info, dict_new_extra_info)
df_combined_extra_info

In [ ]:
#to big for GitHub
#df_combined_extra_info.to_csv('extra_info_CS5099.csv', sep='\t')

Both Dataframes columns are stored each to an object. The series objects are stored to each to a pkl-file which is not exceeding the size of 100MB allowing GitHub uploads.

In [ ]:
# stored_series = store_df_columns(df_combined_extra_info)
# stored_series[0]

In [ ]:
# stored_series[1]

Verifying that the returned None values are due to non existent data and not to an invalid API-Key

In [ ]:
# len_data = len(stored_series[0])
# len_data 

In [ ]:
# ser_doi = pd.Series(doi_counted.index[:len_data])
# ser_doi

In [ ]:
# df_current_extra_info_checker = df_combined_extra_info
# df_current_extra_info_checker['doi'] = ser_doi

In [ ]:
# %%time
# len_df_current_extra_info_checker = len(df_current_extra_info_checker)
# dict_new_extra_info_checker = dict()
# i = 0 
# while i < len_df_current_extra_info_checker: ###################################################### 
#     if df_current_extra_info_checker['affiliation'][i] == None:
#         dict_new_extra_info_checker[i] = fetch_scopus_api(client, ser_doi[i])
#         print("Position fetched again: " + str(i) + " -> " +  ser_doi[i])
#     i = i + 1    

In [ ]:
# dict_new_extra_info_checker
# -> check if at least one value is not None -> otherwise the process is finished here

In [ ]:
# len(dict_new_extra_info_checker)

In [ ]:
# df_combined_extra_info_fetched_again  = append_fetched_data_to_df(df_current_extra_info, dict_new_extra_info_checker)
# df_combined_extra_info_fetched_again

In [ ]:
# store_df_columns(df_combined_extra_info_fetched_again)

In [ ]:
# df_combined_extra_info_fetched_again['check_doi'] = ser_doi
# df_combined_extra_info_fetched_again.head(30)